In [1]:
import os,fnmatch,sys
import shutil,re
import numpy as np
from natsort import natsorted
import cv2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import tifffile as TIF
import time


class ProgressBar(object):
    def __init__(self, message, width=20, progressSymbol=u'⬜', emptySymbol=u'⬛'):
        self.width = width
 
        if self.width < 0:
            self.width = 0
 
        self.message = message
        self.progressSymbol = progressSymbol
        self.emptySymbol = emptySymbol  
    
    def update(self, progress):
        
        totalBlocks = self.width
        filledBlocks = int(round(progress / (100 / float(totalBlocks)) ))
        emptyBlocks = totalBlocks - filledBlocks
 
        progressBar = self.progressSymbol * filledBlocks + \
                      self.emptySymbol * emptyBlocks
  
        if not self.message:
            self.message = u''
  
        progressMessage = u'\r{0} {1}  {2}%'.format(self.message,
                                                    progressBar,
                                                    progress)
 
        sys.stdout.write(progressMessage)
        sys.stdout.flush()
 
 
    def calculateAndUpdate(self, done, total):
        progress = int(round( (done / float(total)) * 100) )
        self.update(progress)
        

def makedirs(dest):
    if not os.path.exists(dest):
        os.makedirs(dest)

        
def listFilesAndCount(directory,pattern = '*'):
    dirList = []
    fileNamelist = []
    fullPathList = []
    if os.path.isdir(directory):       
        for root, dirs, filenames in os.walk(directory):
            dirList.append(root)
            for filename in fnmatch.filter(filenames, pattern):
                fullPathList.append(os.path.join(root, filename))
                fileNamelist.append(filename) 
    return dirList,fileNamelist,fullPathList,len(fullPathList)
 

def exportMDPlate(src, dest, pattern = r'((?<=_w[0-9]_thumb).*(?=.tif))|((?<=_w[0-9])(?!_thumb).*(?=.tif))', moveFlag = False): 
#|((?<=_s[0-9]+_thumb).*(?=.tif))|((?<=_s[0-9])(?!_w[0-9])(?!_thumb).*(?=.tif))|((?<=_[A-Z][0-1][0-9]_thumb).*(?=.tif))|((?<=_[A-Z][0-1][0-9])(?!_s[0-9]+)(?!_w[0-9])(?!_thumb).*(?=.tif))    
    dirList,fileNamelist,fullPathList,numFiles = listFilesAndCount(src)
    
    exportPB = ProgressBar('Copying files...')

    if numFiles > 0:
        makedirs(dest)
 
        numCopied = 0

        for directory in dirList:
            destDir = directory.replace(src,dest)
            makedirs(destDir)
                    
        exportPB.calculateAndUpdate(numCopied, numFiles)            
        for curFile in fullPathList:                                  
           
            #remove the HASH from the full filename
            destFile = curFile.replace(src, dest)
            destFile = re.sub(pattern,'',destFile)   
            if moveFlag:
                shutil.copy2(curFile, destFile)
            else:
                shutil.move(curFile, destFile)
            numCopied += 1
            exportPB.calculateAndUpdate(numCopied, numFiles)        
    else :
        print('No files found. Please check the input path.')
            
def parseImageFileNames(fullPathList,tok_regex = 
                        r'\\(?P<timepoint>TimePoint_\d*)\\.*_(?P<well>[A-Z][0-1][1-9])_(?P<site>s[0-9]+)_(?P<channel>w[0-9]).tif'):
    #Timepoints Wells Sites Channels
  

    timepointList = []
    wellList = []
    siteList = []
    channelList = []
    matchedFileList = []

 
    for iFileName in  fullPathList:

        matches = re.search(tok_regex,iFileName)
       
        if matches is None:
            continue
        
    
        try:
            timepointList.append(matches.group('timepoint'))
        except AttributeError:
            pass
        except IndexError:
            pass
        
        try:
            wellList.append(matches.group('well'))
        except AttributeError:
            pass
        except IndexError:
            pass
        try:
            siteList.append(matches.group('site'))
        except AttributeError:
            pass
        except IndexError:
            pass
        
        try: 
            channelList.append(matches.group('channel'))
        except AttributeError:
            pass
        except IndexError:
            pass
        
        try: 
            matchedFileList.append(matches.string)
        except AttributeError:
            print('No Matches found')   
        except IndexError:
            print('No Matches found')
            
  
        

    timepointList = natsorted(set(timepointList))
    wellList = natsorted(set(wellList))
    channelList = natsorted(set(channelList))
    siteList = natsorted(set(siteList))
    
    return natsorted(matchedFileList), timepointList, wellList, channelList,siteList            
            
            
            
def filterFileList(inputList,filters,filterFlag = 'ALL'):
    filteredList = []
    if filterFlag == 'ANY':
        filteredList = [iFileName for iFileName in inputList if any(iFilter in iFileName for iFilter in filters)]    
    
    if filterFlag == 'ALL':
        filteredList = [iFileName for iFileName in inputList if all(iFilter in iFileName for iFilter in filters)]
    
    return natsorted(filteredList)

def filterFileListRegex(inputList,inputRegex= r'\\(?P<timepoint>TimePoint_\d*)\\.*_(?P<well>[A-Z][0-1][1-9])_(?P<site>s[0-9]+)_(?P<channel>w[0-9]).tif' ,
                        selTP= [],
                        selWell= [],
                        selSite= [],
                        selChannel = []):

    if selTP and type(selTP) is str :
        tpRegex = r'.*\(\?P\<timepoint.*?\)'
        inputRegex = re.sub(tpRegex , selTP , inputRegex)
    
    if selWell and type(selWell) is str :
        wellRegex = r'\(\?P\<well.*?\)'
        inputRegex = re.sub(wellRegex , selWell , inputRegex)
    
    
    if selSite and type(selSite) is list and len(selSite) > 1:
        selSite = '|'.join(selSite)
        selSite = '('+selSite+')'
    
    
    if selSite and type(selSite) is str :  
        siteRegex = r'\(\?P\<site.*?\)'
        inputRegex = re.sub(siteRegex , selSite , inputRegex)
    
    

    if selChannel and type(selChannel) is str :
        channelRegex = r'\(\?P\<channel.*?\)'
        inputRegex = re.sub(channelRegex ,selChannel , inputRegex)

    out = parseImageFileNames(inputList, tok_regex =  inputRegex)
    
    return out[0]


def stitchImages(inputList,nRow,nCol,resizeFactor = 1, padding=0 ,labelImages = False,labelNames = 'default' ,font = cv2.QT_FONT_NORMAL,
                    bottomLeftCornerOfText = (20,20), fontScale = 1, fontColor = 2**16, lineType = 2):

    #read first file and extract raw image dimensions

    firstImage = cv2.imread(inputList[0],0)
    imageY,imageX = firstImage.shape 
    
    #cv2.imshow('First',firstImage)
    iImage = 0

    if resizeFactor != 1:
        reImageY, reImageX = (int(round(imageY * resizeFactor)), int(round(imageX * resizeFactor)))
    else:
        reImageY, reImageX = imageY, imageX

    stitchedY,stitchedX = reImageY*nCol+padding*nCol-padding, reImageX*nRow+padding*nRow-padding



    stitchedImage = np.zeros(shape=(stitchedX,stitchedY), dtype=np.uint16)    
    
    for iRow in range(nRow):
         for iCol in range(nCol):
            if( iImage < len(inputList)):
                  
                curImage = cv2.imread(inputList[iImage],-1)
                curImage = cv2.resize(curImage,(reImageY, reImageX), interpolation = cv2.INTER_AREA)
                
                if(labelImages == True):
                    
                    if(any(labelNames) == 'default'):
                        cv2.putText(curImage,'S'+str(iImage+1),bottomLeftCornerOfText, font,fontScale,fontColor,2,cv2.LINE_AA)
                    else:
                        cv2.putText(curImage,labelNames[iImage],bottomLeftCornerOfText, font,fontScale,fontColor,2,cv2.LINE_AA)
                        
                stitchedImage[iRow*reImageX+iRow*padding:iRow*reImageX+iRow*padding+reImageX,iCol*reImageY+iCol*padding:iCol*reImageY+iCol*padding+reImageY] = curImage
                
                iImage = iImage + 1
                
            else:
                print('WARNING:not enough input images to fill the full stitch grid.')
                break
    
    return stitchedImage


def generateOverviews ( inputFileList,timepointList, wellList, channelList, siteList, outputFilename = 'temp.tif',
                        overRows=1, overCols=2, overResizeFactor = 0.05, overPadding=30 ,
                        siteRows = 1, siteCols = 1, sitePadding = 0,
                        labelImages = True ,labelNames = 'default' ,font = cv2.QT_FONT_NORMAL,
                        bottomLeftCornerOfText = (20,20), fontScale = 0.5, fontColor = 2**16, lineType = 2, overwriteFlag = True,
                        inputRegex= r'\\(?P<timepoint>TimePoint_\d*)\\.*_(?P<well>[A-Z][0-1][1-9])_(?P<site>s[0-9]+)_(?P<channel>w[0-9]).tif'):
    
    iProgress = 0
    overviewPB = ProgressBar('Generating Overview...')
    
    
    if os.path.exists(outputFilename) & (not overwriteFlag):
         print('Output file already exists')
         return 0
     
    if not inputFileList:
        print('\r ERROR: No images found')
        return

    if type(inputFileList) == str:
        inputFileList = [inputFileList]   
    
    if type(timepointList) == str:
        timepointList = [timepointList]   

    if type(wellList) == str:
        wellList = [wellList]   
        
    if type(channelList) == str:
        channelList = [channelList]  
        
    if type(siteList) == str:
        siteList = [siteList]
        
    # if timepoint list is empty subsitute it by '1'
    if timepointList == [] :
        timepointList = '1'
       
    
    numberOfGridEllements = len(channelList)*len(timepointList)*overCols*overRows
    overviewPB.calculateAndUpdate(iProgress, numberOfGridEllements)       
    
        
    for iTp in range(len(timepointList)):
    
        if (iTp == 0):
                
            
                #initialize


                      
                firstImage = cv2.imread(inputFileList[0],0)
                imageY,imageX = firstImage.shape                
                reImageY, reImageX = (int(round(imageY * overResizeFactor)), int(round(imageX * overResizeFactor)))                
                imageY,imageX  = reImageY*siteCols+sitePadding*siteCols-sitePadding, reImageX*siteRows+sitePadding*siteRows-sitePadding
                
                
    #             print(imageY,imageX)
                overY,overX = imageY*overCols+overPadding*overCols-overPadding, imageX*overRows+overPadding*overRows-overPadding
                overviewImage = np.zeros(shape=(len(timepointList),1,len(channelList),overX,overY), dtype=np.uint16)   #'TZCYX'


        for iChannel in range(len(channelList)):   

            iWell = 0

            for iRow in range(overRows):
                for iCol in range(overCols):   
                    
                    
                    overviewPB.calculateAndUpdate(iProgress, numberOfGridEllements)
                    iProgress = iProgress +1 
                    if( iWell < len(wellList)):
                        if timepointList == '1':
                            filteredList = filterFileListRegex(inputFileList, selTP= timepointList[iTp], selWell= wellList[iWell], selSite = siteList,
                                                               selChannel = channelList[iChannel],
                                                               inputRegex=inputRegex)
                        else:
                                                       
                            if len(siteList) == 1:
                                filteredList = filterFileListRegex(inputFileList,selTP= timepointList[iTp],selWell= wellList[iWell], selSite = siteList[0],selChannel = channelList[iChannel],
                                                                   inputRegex=inputRegex)
                            else:
                                filteredList = filterFileListRegex(inputFileList,selTP= timepointList[iTp],selWell= wellList[iWell], selSite = siteList,selChannel = channelList[iChannel],
                                                                   inputRegex=inputRegex)
                        
                        if not filteredList:
                            print('\rWARNING: No images found at Timepoint: '+timepointList[iTp]+' Well: '+ wellList[iWell]+' Channel: '+channelList[iChannel])
                            overviewPB.calculateAndUpdate(iProgress, numberOfGridEllements)
                            continue
                        #Include only sites included in the sitelist
#                         filteredList = filterFileList(filteredList,siteList,filterFlag = 'ANY')
                        
#                         print(filteredList)
                            
                        stitchedWell = stitchImages(filteredList,siteRows,siteCols,overResizeFactor,sitePadding,labelImages = labelImages,
                                                    labelNames = np.core.defchararray.add(wellList[iWell]+'_', siteList),
                                                    font =font, bottomLeftCornerOfText = bottomLeftCornerOfText, fontScale = fontScale,
                                                    fontColor= fontColor, lineType = lineType)
                            
                        overviewImage[iTp,0,iChannel,iRow*imageX+iRow*overPadding:iRow*imageX+iRow*overPadding+imageX,iCol*imageY+iCol*overPadding:iCol*imageY+iCol*overPadding+imageY] = stitchedWell

                        iWell = iWell + 1


                    else:
                       # print('WARNING: Not enough input images to fill the overview grid.')
                        continue
    print('\n\rSaving the generated image...')
    TIF.imsave(outputFilename, overviewImage,imagej=True,metadata={'axes': 'TZCYX'})                
    print('\rDone')
    return overviewImage

In [ ]:

dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(r'W:\Vardan_Andriasyan\180629-MB-VA-A549-HSV')

matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList)

In [ ]:
#Exporting a single 96-well plate

inputPath = r'Z:\Anthony_Petkidis\experiments\190826-AP-Hoechst\190826-AP-Hoechst\2019-08-26\14168'

outputPath = r'Z:\Anthony_Petkidis\experiments\190826-AP-Hoechst\14167'

exportMDPlate(inputPath,outputPath,moveFlag = False)


In [ ]:
#Overview generation
inputPath = r'Z:\Anthony_Petkidis\experiments\190826-AP-Hoechst\14167'


dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(inputPath)
# print(fullPathList)
# token_spec = [(r'.*'),  #_\d*(r'\\(?P<timepoint>TimePoint_\d+)\\(?!Stitched)'),  #_\d*
#      (r'(?P<well>[A-Z][0-1][1-9])'),         
#      (r'_(?P<site>s[0-9]+)'),  
#      (r'(?P<channel>w[0-9]).tif')]
tok_regex  = r'.*_(?P<well>[A-Z][0-1][1-9])_(?P<site>s[0-9]+)_(?P<channel>w[0-9]).tif'
# outpath = r'Z:\Anthony_Petkidis\experiments\190826-AP-Hoechst\hoechst_control_14167.tif'
matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList,tok_regex  = r'.*_(?P<well>[A-Z][0-1][1-9])_(?P<site>s[0-9]+)_(?P<channel>w[0-9]).tif')

over = generateOverviews (matchedFileList,timepointList, wellList, channelList, siteList,
                          overRows=8, overCols=4, overResizeFa+
                          ctor = 0.25, overPadding=50,
                          siteRows = 2, siteCols = 2 , 
                          outputFilename = outpath , labelImages = True, overwriteFlag= True, inputRegex=tok_regex )

# # print(wellList)

In [2]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
#change display settings to fit pandas tables in output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


file = r'W:\Vardan_Andriasyan\POSTPROCESSING\annotation_sheet.csv'
df = pd.read_csv(file)

	
filteredDF = df[df.Source_ID.str.contains('TC6',case=False)]


wellsToStack = list((filteredDF['Well_ID'].values))

#parse input folder and get all relevant files
inputPath = filteredDF.Source_path[0]
dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(inputPath)
matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList)



In [4]:
inputPath = r'W:\Vardan_Andriasyan\180629-MB-VA-A549-HSV'
dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(inputPath)
matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList)


In [11]:
parentFolder = r'W:\Vardan_Andriasyan\POSTPROCESSING\HSV_stitches'

inputRegex= r'\\(?P<timepoint>TimePoint_\d*)\\.*_(?P<well>[A-Z][0-1][1-9])_(?P<site>s[0-9]+)_(?P<channel>w[0-9]).tif'

wellsToStack = ['B02_s26']
ind = 0
for iWellID in wellsToStack:
   
    wellName,siteName = str.split(iWellID,'_')
    outpath = os.path.join(parentFolder,iWellID+'.tif')
#     filteredDF.loc[[ind],['Stitch_path']] = outpath
    print('Processing:'+iWellID+'...') 
    over = generateOverviews(matchedFileList,timepointList, wellName, channelList, siteName,  
        overRows=1, overCols=1, overResizeFactor = 1, overPadding=0,
        siteRows = 1, siteCols = 1 , 
        outputFilename = outpath , labelImages = False, overwriteFlag = False ,inputRegex = inputRegex)

    ind = ind+1
    
    

Processing:B02_s26...
Generating Overview... ⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜  100%
Saving the generated image...
Done


In [91]:
import xml

# doc = xml.dom.minidom.parse(r'W:\Vardan_Andriasyan\POSTPROCESSING\HADV_TC6_stitched\tracks\B02_s33.xml')

doc = xml.dom.minidom.parse(r'W:\Vardan_Andriasyan\POSTPROCESSING\HADV_TC6_stitched\tracks\B02_s33.xml')

tracks = doc.documentElement
AA = tracks.getElementsByTagName("Track")

for itrack in AA:
    print(itrack.getAttribute("Edge"))
# for iTrack in tracks:
print(AA.getAttribute("title"))
# print(doc.getElementsByTagName('Track').item(1).i)

AttributeError: 'NodeList' object has no attribute 'getAttribute'

In [4]:
import keras
print(keras.__version__)
import tensorflow as tf
print(tf.__version__)

2.2.4
1.13.1


In [9]:
import xml.etree.ElementTree as ET
tree = ET.parse(r'W:\Vardan_Andriasyan\POSTPROCESSING\HADV_TC6_stitched\tracks\B02_s33.xml')
root = tree.getroot()
for neighbor in root.iter('Edge'):
         print(neighbor.attrib)
# print(tree.findall('xml'))
# root.getchildren
# for elem in root:
#     for subelem in elem:
#         print(subelem.)

{'SPOT_SOURCE_ID': '257', 'SPOT_TARGET_ID': '260', 'LINK_COST': '-1.0', 'EDGE_TIME': '0.0', 'EDGE_X_LOCATION': '784.5', 'EDGE_Y_LOCATION': '757.8333333333334', 'EDGE_Z_LOCATION': '0.0', 'VELOCITY': 'Infinity', 'DISPLACEMENT': '2.403700850309284'}
{'SPOT_SOURCE_ID': '266', 'SPOT_TARGET_ID': '269', 'LINK_COST': '-1.0', 'EDGE_TIME': '0.0', 'EDGE_X_LOCATION': '825.8333333333334', 'EDGE_Y_LOCATION': '758.1666666666667', 'EDGE_Z_LOCATION': '0.0', 'VELOCITY': 'Infinity', 'DISPLACEMENT': '22.95890047696349'}
{'SPOT_SOURCE_ID': '203', 'SPOT_TARGET_ID': '206', 'LINK_COST': '-1.0', 'EDGE_TIME': '0.0', 'EDGE_X_LOCATION': '801.1666666666667', 'EDGE_Y_LOCATION': '697.1666666666666', 'EDGE_Z_LOCATION': '0.0', 'VELOCITY': 'Infinity', 'DISPLACEMENT': '10.088497300281027'}
{'SPOT_SOURCE_ID': '224', 'SPOT_TARGET_ID': '227', 'LINK_COST': '-1.0', 'EDGE_TIME': '0.0', 'EDGE_X_LOCATION': '800.1666666666666', 'EDGE_Y_LOCATION': '714.5', 'EDGE_Z_LOCATION': '0.0', 'VELOCITY': 'Infinity', 'DISPLACEMENT': '25.1572

In [ ]:
import numpy as np
from natsort import natsorted
import matplotlib
from matplotlib import pyplot as plt
import time
import tifffile as TIF
import shutil

%matplotlib qt


# src = r'W:\Anthony_Petkidis\181127-AP-dE3B-TC1\181127-AP-dE3B-TC1\2018-11-27\123'
# dest = r'W:\Vardan_Andriasyan\180307-VA-dE3B-TC-7' 



src =  r'W:\Vardan_Andriasyan\180629-MB-VA-A549-HSV'
src = r'W:\Vardan_Andriasyan\180315-VA-dE3B-dADP-TC-8'
src = r'W:\Vardan_Andriasyan\180307-VA-dE3B-TC-7' 
src = r'W:\Vardan_Andriasyan\170719-VA-E3-E1-PV-TC' 
src = r'W:\Vardan_Andriasyan\180315-VA-dE3B-dADP-TC-8'
src = r'W:\Vardan_Andriasyan\180307-VA-dE3B-TC-7' 
src = r'W:\Vardan_Andriasyan\180301-VA-dE3B-TC-6' 
src = r'W:\Vardan_Andriasyan\171208-VA-PV-oldA549-TC-5' 
# exportMDPlate(src,dest)
 
dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(src)


# # src = r'W:\Vardan_Andriasyan\170719-VA-E3-E1-PV-TC\170719-VA-E3-E1-PV-TC\2017-07-19\414'
# # dest = r'W:\Vardan_Andriasyan\170728-VA-E3-E1-PV-TC-2' 

# # exportMDPlate(src,dest)

# shutil.rmtree(r'W:\Vardan_Andriasyan\170719-VA-E3-E1-PV-TC\170719-VA-E3-E1-PV-TC\2017-07-19')


# stitchedWell = stitchImages(inputList,siteRows,siteCols,resizeFactor,padding,labelImages = True)
# plt.imshow(stitchedWell,vmin = 0, vmax = 2000,  c map='gray')
# plt.show()


# data = np.random.randint(0, 2**16, (4,301, 219,5), 'uint16')
# # TIF.imsave('temp.tif', data, photometric='minisblack')

# timepointList = ['TimePoint_57']+
# wellList = ['B02']

# dest = r'W:\Vardan_Andriasyan\170916-VA-PV-oldA549-TC-4' 
 

In [26]:

# token_spec = [(r'\\(?P<timepoint>TimePoint_\d*)\\(?!Stitched)'),  2
#     (r'(?P<well>[A-Z][0-1][1-9])'),         
#     (r'_(?P<site>s[0-9]+)'), 
#     (r'(?P<channel>w[0-9]).tif')]

token_spec = [(r'\\(?P<timepoint>TimePoint_\d+)\\(?!Stitched)'),  #_\d*
     (r'(?P<well>[E][0][3])'),         
     (r'_(?P<site>s16)'), 
     (r'(?P<channel>w[0-9]).tif')]


curPath = r'W:\Vardan_Andriasyan\POSTPROCESSING\forFG\E03_s16.tif'
 
matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList,token_specification = token_spec)

over = generateOverviews (matchedFileList,timepointList, wellList, channelList, siteList,
                          overRows=1, overCols=1, overResizeFactor = 1, overPadding=0,
                          siteRows = 1, siteCols = 1, 
                          outputFilename = curPath , labelImages = False )

TypeError: parseImageFileNames() got an unexpected keyword argument 'token_specification'

In [ ]:
import numpy as np
from natsort import natsorted
import matplotlib
from matplotlib import pyplot as plt
import time
import tifffile as TIF
import shutil

%matplotlib qt


# src = r'W:\Anthony_Petkidis\181127-AP-dE3B-TC1\181127-AP-dE3B-TC1\2018-11-27\123'
# dest = r'W:\Vardan_Andriasyan\180307-VA-dE3B-TC-7' 



src =  r'W:\Vardan_Andriasyan\180629-MB-VA-A549-HSV'
src = r'W:\Vardan_Andriasyan\180315-VA-dE3B-dADP-TC-8'
src = r'W:\Vardan_Andriasyan\180307-VA-dE3B-TC-7' 
src = r'W:\Vardan_Andriasyan\170719-VA-E3-E1-PV-TC' 

# exportMDPlate(src,dest)



dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(src)

# token_spec = [(r'\\(?P<timepoint>TimePoint_\d*)\\(?!Stitched)'),  2
#     (r'(?P<well>[A-Z][0-1][1-9])'),         
#     (r'_(?P<site>s[0-9]+)'), 
#     (r'(?P<channel>w[0-9]).tif')]

token_spec = [(r'\\(?P<timepoint>TimePoint_\d+)\\(?!Stitched)'),  #_\d*
     (r'(?P<well>F02)'),         
     (r'_(?P<site>s8)'), 
     (r'(?P<channel>w[0-9]).tif')]


curPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\tc1_pV_F02_s8.tif'


matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList,token_specification = token_spec)

over = generateOverviews (matchedFileList,timepointList, wellList, channelList, siteList,
                          overRows=1, overCols=1, overResizeFactor = 1, overPadding=0,
                          siteRows = 1, siteCols = 1, 
                          outputFilename = curPath , labelImages = False )

# # src = r'W:\Vardan_Andriasyan\170719-VA-E3-E1-PV-TC\170719-VA-E3-E1-PV-TC\2017-07-19\414'
# # dest = r'W:\Vardan_Andriasyan\170728-VA-E3-E1-PV-TC-2' 

# # exportMDPlate(src,dest)

# shutil.rmtree(r'W:\Vardan_Andriasyan\170719-VA-E3-E1-PV-TC\170719-VA-E3-E1-PV-TC\2017-07-19')



# stitchedWell = stitchImages(inputList,siteRows,siteCols,resizeFactor,padding,labelImages = True)
# plt.imshow(stitchedWell,vmin = 0, vmax = 2000,  c map='gray')
# plt.show()


# data = np.random.randint(0, 2**16, (4,301, 219,5), 'uint16')
# # TIF.imsave('temp.tif', data, photometric='minisblack')

# timepointList = ['TimePoint_57']+
# wellList = ['B02']

# dest = r'W:\Vardan_Andriasyan\170916-VA-PV-oldA549-TC-4' 
 

In [ ]:
import file_utils

%matplotlib qt


src = r'W:\Anthony_Petkidis\181127-AP-dE3B-TC1\181127-AP-dE3B-TC1\2018-11-27\123'
dest = r'W:\Vardan_Andriasyan\180307-VA-dE3B-TC-7' 


exportMDPlate(src,dest)


dirList,fileNamelist,fullPathList,fileNum = file_utils.listFilesAndCount(dest)

token_spec = [(r'\\(?P<timepoint>TimePoint_\d*)\\'),  
    (r'(?P<well>[A-Z][0-1][1-9])'),         
    (r'_(?P<site>s[0-9]+)'), 
    (r'(?P<channel>w[0-9]).tif')]


outPath = r'W:\Vardan_Andriasyan\POSTPROCESSING\racaniello_pres\res\hsv_C02_s20.tif'


matchedFileList, timepointList, wellList, channelList,siteList  = file_utils.parseImageFileNames(fullPathList,token_specification = token_spec)

over = file_utils.generateOverviews (matchedFileList,timepointList, wellList, channelList, siteList,
                          overRows=1, overCols=1, overResizeFactor = 0.05, overPadding=30,
                          siteRows = 1, siteCols = 1, 
                          outputFilename = outPath , labelImages = False )


# # src = r'W:\Vardan_Andriasyan\170719-VA-E3-E1-PV-TC\170719-VA-E3


HAdV-dE3b-GFP ::::::  TC7 G03_s11_T28,G03_s12_T28 , E03_s16_T28, E02_s40_T28
HAdV-WT ::::::
HSV ::::::  


In [ ]:

# READ AND LIST THE PARENT FOLDER
src = r'W:\Anthony_Petkidis\190115-AP-dE3B-TC2'
src = r'W:\Anthony_Petkidis\190115-AP-dE3B-fixed-nonimaged-TC2'
dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(src)


token_spec = [(r'_(?P<well>D[0-1][0-9])_'),
              (r'(?P<site>s[0-9]+)'), 
            (r'_(?P<channel>w[0-9])_'),
              (r'(?P<timepoint>T1_Z5.tif)')]

matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList,token_specification = token_spec)
print(channelList)
outPath = r'W:\Vardan_Andriasyan\Lamin_over_lasttp.tif'
over = generateOverviews(matchedFileList,timepointList, wellList, channelList, siteList,
                          overRows=1, overCols=8, overResizeFactor = 0.25, overPadding=30,
                          siteRows = 7, siteCols = 7, 
                          outputFilename = outPath , labelImages = True )

# wellList = ['B03']

# outPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES'

# for iWell in wellList:
    
#     token_spec = [(r'\\(?P<timepoint>TimePoint_43)\\'),  
#     ('(?P<well>'+ iWell+')'),         
#     (r'_(?P<site>s[0-9]+)'), 
#     (r'(?P<channel>w[0-9]).tif')]
            
#     curName = iWell +'.tif'
#     curName = 'dE3B_raw.tif'
#     outName = os.path.join(outPath,curName)
    
#     matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList,token_specification = token_spec)
#     #siteList = ['s10','s11','s12','s17','s18','s19']  WT well b02 src = r'W:\Vardan_Andriasyan\180910-VA-WT-TC18-IMAGED-FIX'
#     siteList = ['s11','s12','s18','s19','s25','s26']
    
# #     siteList= ['s8','s9']
    
#     over = generateOverviews (matchedFileList,timepointList, wellList, channelList, siteList,
#                           overRows=1, overCols=1, overResizeFactor = 1, overPadding=0,
#                           siteRows = 3, siteCols = 2, 
#                           outputFilename = outName , labelImages = False )
    #print(matchedFileList)


In [ ]:
# print(fullPathList)
siteList = ['s1','s20']
filters= ['E02','w1.tif']
filteredList = filterFileList(fullPathList,filters,filterFlag = 'ALL')

filters= ['_s1_','_s3_']
filteredList2 = filterFileList(filteredList,siteList,filterFlag = 'ANY')

print(filteredList)

In [ ]:
print(fullPathList[0]) 

tok = [(r''),  
     (r'_(?P<well>[A-Z][0-1][1-9])'),         
     (r'_(?P<site>s[0-9]+)'), 
     (r'(?P<channel>w[0-9]).tif')]

matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList)
print(timepointList, wellList, channelList,siteList)

In [ ]:

dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(r'W:/Vardan_Andriasyan/test5/')

m, T, W, C, S  = parseImageFileNames(fullPathList, token_specification = [(r''),
                                                                          (r'(?P<well>[A-Z][0-1][1-9])'),
                                                                          (r''), 
                                                                          (r'')])
  
    
print(T)


In [ ]:
a = np.array([1,2],[1,2])
print(a[1,2])

In [ ]:
import time

src = r'W:\Vardan_Andriasyan\180817-VA-AD2WT-TC-14-FIX-ALTER\backup\2018-08-22\58'
dest = r'W:\Vardan_Andriasyan\180817-VA-AD2WT-TC-14-FIX-ALTER'

src = r'W:\Vardan_Andriasyan\test3'
dest = r'W:\Vardan_Andriasyan\deleteme'

src = r'W:\Vardan_Andriasyan\180910-VA-WT-TC18-IMAGED-FIX\180910-VA-WT-TC18-IMAGED-FIX\2018-09-24\67'
dest = r'W:\Vardan_Andriasyan\180910-VA-WT-TC18-IMAGED-FIX\180910-VA-WT-TC18-IMAGED-FIX'

start = time.time()
exportMDPlate(src,dest)
end = time.time()

#print('\n'+str((end - start)/60/60))
             


In [ ]:
   #name = input("What is your name? ")
#print (name) 
test = 0.656456461

print('this is a test number {0:.3f}'.format(test))

testName = r'W:\Vardan_Andriasyan\180705-VA-AD2WT-TC-11\TimePoint_1\180705-VA-AD2WT-TC-11_B02_s1_w1.tif'

dirList,fileNamelist,fullPathList,fileNum = listFilesAndCount(r'W:\Vardan_Andriasyan\testSet\TimePoint_1')
# print(fullPathList)




In [ ]:
Z = ['A01','H12','B11','C06']
# print(natsorted(Z))
# print(fullPathList)
2**2

In [ ]:

testName = r'W:\Vardan_Andriasyan\180705-VA-AD2WT-TC-11\TimePoint_1\180705-VA-AD2WT-TC-11_B02_s1_w1.tif'

tok_regex = '_B02_'
tok_regex = '.*(?P<site>s[0-9] +).*(?P<channel>w[0-9])'
matches = re.search(tok_regex,testName)

print(matches)

# out = re.search(tok_regex,testName)
# print(out.string)

# print(fullPathList)

In [ ]:

dirList1,fileNamelist1,fullPathList1,fileNum1 = listFilesAndCount(r'W:\Vardan_Andriasyan\180910-VA-WT-TC18-IMAGED-FIX\180910-VA-WT-TC18-IMAGED-FIX')
    
matchedFileList, timepointList, wellList, channelList,siteList  = parseImageFileNames(fullPathList1)

In [ ]:
print(wellList)

In [ ]:

stitchedWell = stitchImages(inputList,siteRows,siteCols,resizeFactor,padding,labelImages = True)
plt.imshow(stitchedWell,vmin = 0, vmax = 2000,  cmap='gray')
plt.show()


In [ ]:
print(stitchedWell.shape)

In [ ]:
print(len(channelList)*len(timepointList)*len(wellList))
print(iImage)

In [ ]:
A = TIF.imread("W:\Vardan_Andriasyan\POSTPROCESSING\single_site\AGGR\Composite.tif")
A.shape = 58,1,3, 2048, 2048

# 1, 57, 1, 256, 256, 1   
TIF.imsave('temp.tif', A,imagej=True,metadata={'axes': 'TZCYX'})
# print(np.max(A[:]))


In [ ]:
# plt.imshow(overviewImage,vmin = 0, vmax = 2000,  cmap='gray') 
# plt.show()
              
                
                
#             firstImage = cv2.imread(plateFileList[0],0)
#             imageY,imageX = firstImage.shape 
#             cv2.imshow('First',firstImage)            
        
#             if resizeFactor != 1:
#                 reImageY, reImageX = (int(round(imageY * resizeFactor)), int(round(imageX * resizeFactor)))
#             else:
#                 reImageY, reImageX = imageY, imageX
#         filters= [iTp,iWell,iChannel+'.tif']
#         inputList = filterFileList(matchedFileList,filters)
#         stitchedWell = stitchImages(inputList,siteRows,siteCols,resizeFactor,padding,labelImages = False)
        

        
        
        
                #read first file and extract raw image dimensions

#             firstImage = cv2.imread(inputList[0],0)
#             imageY,imageX = firstImage.shape 

#             #cv2.imshow('First',firstImage)            

#             if resizeFactor != 1:
#                 reImageY, reImageX = (int(round(imageY * resizeFactor)), int(round(imageX * resizeFactor)))
#             else:
#                 reImageY, reImageX = imageY, imageX
 
#             stitchedY,stitchedX = reImageY*nCol+padding*nCol-padding, reImageX*nRow+padding*nRow-padding

#             #IF THE IMAGES ARE STITCHED
            
            
            
            
            
            
#             #IF THE IMAGES ARE NOT PRESTITCHED
            
            
#             for iRow in range(overRows)
#                 for iCol in range(overCols)
                    
#                     curImage = cv2.imread(inputList[iImage],-1)
#                     curImage = cv2.resize(curImage,(reImageY, reImageX), interpolation = cv2.INTER_AREA)
                    
                    
                    
#                     if(labelImages == True):
#                         font = cv2.QT_FONT_NORMAL
#                         bottomLeftCornerOfText = (50,50)
#                         fontScale = 2
#                         fontColor = 2**16
#                         lineType = 2
#                         if(labelNames == 'default'):
#                             cv2.putText(curImage,'S'+str(iImage+1),bottomLeftCornerOfText, font,fontScale,fontColor,2,cv2.LINE_AA)
#                         else:
#                             cv2.putText(curImage,labelNames[iImage],bottomLeftCornerOfText, font,fontScale,fontColor,2,cv2.LINE_AA)

#                     stitchedImage[iRow*reImageX+iRow*padding:iRow*reImageX+iRow*padding+reImageX,iCol*reImageY+iCol*padding:iCol*reImageY+iCol*padding+reImageY] = curImage

#                     iImage = iImage + 1         
                    
                    
#         print(timepointList[iTp] + ' ' + iChannel)
    
#             iWell  = 'B02'
#             filters= ['TimePoint_'+iTp,iWell,'_'+iChannel+'.tif']
#             inputList = filterFileList(matchedFileList,filters)
#             stitchedWell = stitchImages(inputList,siteRows,siteCols,resizeFactor,padding,labelImages = False)
#             cv2.imwrite(outputfolder+'\\'+iWell+'_'+'T'+iTp+'_'+iChannel+'.tif', stitchedWell)   

        
#cv2.imwrite(os.path.join(pathToImages, os.pardir, 'stitches', 'plateOverview_t%s_w%s.TIF' % (timepointNo, (w+1))), result)        
        
# Overview Output dir




#cv2.imwrite('testS.TIF', stitchedWell)       
#cv2.imshow('Image',stitchedWell)   
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#cv2.waitKey(0)
#     stitchedWell 
    
    
# # calculate final (stitched) image size, add 2 pixels of empty space per well7
# result_width = int(width * ncol * math.sqrt(nSites) + 2*1/resizeRatio * (ncol-1))
# result_height = int(height * nrow * math.sqrt(nSites) + 2*1/resizeRatio * (nrow-1))
# # generate empty array with the final image size
# result = np.zeros(shape=(result_height, result_width), dtype=np.uint16)






# resizeFactor = 1
# imageSizeY = 80*resizeFactor
# imageSizeX = 64*resizeFactor

# numRows = 7
# numCols = 7




# timepointList = natsorted(set(timepointList))
# wellList = natsorted(set(wellList))
# siteList = natsorted(set(siteList))
# channelList = natsorted(set(channelList))

# print(wellList)


# fullPathList = natsorted(np.array(fullPathList))
# print(fullPathList)

# for iTimepoint in timepointList: 
#     for iChannel in channelList: 
#         for iWell in wellList: 
#             for iSite in siteList: 
#                 []
                


                
# print('Timepoint_'+iTimepoint+'.*'+iWell+'_'+iSite+'_'+iChannel+'.tif')
                
#     for out in re.finditer(tok_regex,iFileName):   
#         print(out)
#         tokenName.append(out.lastgroup)
#         tokenValue.append(out.group(out.lastgroup))
#         # out = re.findall(testPattern,fullPathList[1])
#     tokenValue.insert(0,iFileName)    
#     allValues.append(tokenValue)

    
    
# npVal = np.array(allValues)
    
    
# print(allValues)

    

In [ ]:
# np1 = np.array([[1,2,4],[2,2,2]])
s =  np.zeros((2,2,2))
s[0,0,1] =1
s[1,1,1] =1
print(s[1])


In [ ]:
aaa = 5


In [ ]:
if(bbb):
    pass

In [ ]:
print(matches.groupdict()['timepoint'])

In [ ]:
from natsort import natsorted


npVal = np.array(allValues)

print(all)

# #1print(s[1][1])





# curFilePattern = '_' '_' '_'

# curFilePath =  
S =  (fullPathList,'B03')>-1
# re.search
print(fullPathList[np.core.defchararray.find(fullPathList,'B03')>-1])


listFilesAndCount(directory,'B'):

# print(natsorted(npVal))

In [ ]:
#Define 96 well plate

COLLUMNS = np.arange(1,13)
tmp = COLLUMNS
COLLUMNS  = COLLUMNS.astype('str')
COLLUMNS[tmp<10] = np.core.defchararray.add('0', COLLUMNS[tmp<10])

ROWS =  np.array(['A','B','C','D','E','F','G','H'])
C = np.tile(COLLUMNS,(ROWS.size,1))
R = np.transpose(np.tile(ROWS,(COLLUMNS.size,1)))

WELL_MAP = (np.core.defchararray.add(R,C))
# print(np.asarray(WELL_MAP))


# VALUES = np.full(WELL_MAP.shape, False, dtype=bool)

# plate = {np.asarray(WELL_MAP), np.asarray(VALUES)}
# print(plate
Z = WELL_MAP.ravel()
print(Z)
print(Z[1][2])

In [ ]:
import cv2


tok_regex = r'.*TimePoint_1\\.*_B02_s[0-9]+_w1_thumb.tif'
tokenName = []
tokenValue = []
allValues = []


for iFileName in  fullPathList:
    out = re.search(tok_regex,iFileName)
    if  out is not None:
        allValues.append(out.string)

npVal = natsorted(np.array(allValues))       
#print(npVal)

#def stichSites(siteList=siteList, channelList=channelList ,ncol=ncol, nrow=nrow, resizeRatio=0.1):

resizeFactor = 1
imageSizeY = 80*resizeFactor
imageSizeX = 64*resizeFactor

numRows = 7
numCols = 7


stitchedWell = np.zeros(shape=(64*numRows, 80*numCols), dtype=np.uint16)


# def stitchWell 


#print(np.shape(stitchedWell))

#image = cv2.imread(npVal[0])

#print(image.shape)
i = 0

# for sRow in range(numRows):
#      for sCol in range(numCols):
#         print(sRow + sCol)
#         stitchedWell[sRow*imageSizeX:imageSizeY*sCol] = npVal[i]
#         cv2.imshow('image',cv2.fromarray(stitchedWell))
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
#         i = i+1
        

        
# #imwrite        
# cv2.imshow('image',stitchedWell)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
#Z= np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
#print(Z.shape)
# cv2.imshow('image',cv2.(Z))
# print(Z[0:,0:2])


# well_col = np.array(['1','2','3','4'])

# well_row = np.array(['A','B','C','D'])

# well_col1 = np.array(['1','2','3','4'],dtype = np.object)

# well_row1 = np.array(['A','B','C','D'],dtype = np.object)


# #print(B)
# start = time.time()
# print(start)
# C = well_col1 + well_row1
# end = time.time()
# print(end)


# start = time.time() 
# print(start)
# C = list(map(''.join, zip(well_col,well_row)))
# end = time.time() 
# print(end)

# start = time.time() 
# print(start)
# C = np.core.defchararray.add(well_col, well_row)
# end = time.time() 
# print(end)
#print(C)

#  for y in range(int(math.sqrt(nSites))):
#     for x in range(int(math.sqrt(nSites))):
#         image = cv2.imread(orderedList[i], 2)
#         image = cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)
#         result[int(height * row * int(math.sqrt(nSites)) + (height * y) + (2*1/resizeRatio * row)) : int(height * row * int(math.sqrt(nSites)) + (height * (y + 1)) + (2*1/resizeRatio * row)),
# #         int(width * well * math.sqrt(nSites) + (width * x) + (2*1/resizeRatio * well)) : int(width * well * math.sqrt(nSites) + (width * (x + 1)) + (2*1/resizeRatio * well))] = image
# from skimage import data
# from skimage.viewer import ImageViewer


In [ ]:
testName = '180515-MB-A549-HSV_D02_s1_w1_.tif'
testThumbName = '180301-VA-dE3B-TC-6_G02_s1_w1_thumb179B67-C917D9449656.tif'
testNameClean = '180301-VA-dE3B-TC-6_G02_s1_w1.tif'

In [ ]:
import re
stringNorm = testName
stringThumb = testThumbName

pattern = r'((?<=w[0-9]_thumb).*(?=.tif))|((?<=w[0-9])(?!_thumb).*(?=.tif))'

currentMatch = re.sub(pattern,'',stringThumb)
print(currentMatch)
currentMatch = re.sub(pattern,'',stringNorm)
print(currentMatch)


#if currentMatch is not None:
 #    string = re.sub()   
 




#theMATCH = re.sub("\[[^w[0-9]]*.tif", lambda x:x.group(0).replace(',',''), Variable)



In [ ]:
import pims

v = pims.TiffStack(r'W:\Vardan_Andriasyan\test2\test_Stack.tif')
print(type(v))

In [ ]:

import os,fnmatch
import shutil

filepath = r'180301-VA-dE3B-TC-6_G02_s1_w1171C805A-178E-450D-9B67-C917D9449656.tif'




c(filepath)

In [ ]:
from math import sqrt
from joblib import Parallel, delayed
  
# single-core code
sqroots_1 = [sqrt(i ** 2) for i in range(10)]

# parallel code 
sqroots_2 = Parallel(n_jobs=-1,verbose=5)(delayed(sqrt)(i ** 2) for i in range(10))


def parallelCopyAndRenameTIFs(src, dest,):
    pattern = '*.tif'
         for path, dirs, filenames in os.walk(src):
            for directory in dirs:
                destDir = path.replace(src,dest)
                makedirs(os.path.join(destDir, directory))
            
            for sfile in  fnmatch.filter(filenames, pattern):
                srcFile = os.path.join(path, sfile)
                
                
                
                if len(sfile) > minLength: bu
                    sfile = sfile[0:len(sfile)-40]+'.tif'
                
                destFile = os.path.join(path.replace(src, dest), sfile)
                
                shutil.copy2(srcFile, destFile)
                
                numCopied += 1
                
                p.calculateAndUpdate(numCopied, numFiles)
        print        
        

          



In [ ]:
print(sqroots_2)

In [ ]:
from joblib import Parallel, delayed
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

def myfun(x):
    return x**2

results = Parallel(n_jobs=-1)(delayed(myfun)(i) for i in tqdm(range(1000)))
                              